![Podmanlogo](Pictures/podman-logo.png)

# Managing pods

All this time we've been working with simple containers, but Podman received its name because it's also a pod manager. But, what are a pods? Pods are a group of one or more containers sharing the same network, pid and ipc namespaces. If two containers are in the same pod they can reach each other by using localhost or 127.0.0.1 adress.

Moreover, pods are the smaller atomic unit that kubernetes clusters can manage. Running pods is super useful to run local tests without the need of deploying them on a kubernetes cluster.

Let's see how to manage pods with Podman.

First we are going to create a pod, for the moment it'll be empty.

In [ ]:
podman pod create --name my-pod

We can now see the list of our pods.

In [ ]:
podman pod ls

We can see the list of containers and what pod they are in by running the following command:

In [ ]:
podman ps -a --pod

As you can see there is already a container within our pod. This is the infra container, an infra container is a lightweight container used to coordinate the shared kernel namespace of a pod. This container is created by default, you can overwrite this behaviour but it's not a recommended behaviour.

We will run a wordpress workload within our pod, we will need to run a mariadb container as backend and a wordpress container as frontend. Let's first deploy the database:

In [ ]:
podman run -d --pod my-pod \
  -e MYSQL_ROOT_PASSWORD="hpedev" \
  -e MYSQL_DATABASE="wp" \
  -e MYSQL_USER="wordpress" \
  -e MYSQL_PASSWORD="wordpress" \
  --name=wp-db \
  docker.io/library/mariadb:latest

There are a couple of important points to mention regarding previous command. First, we have specified that our container should be part of our pod with the option "--pod". Second we are passing environment variables to the container with the option "-e", by doing this we can do some basic configuration for this instance of the container.

If we list the containers and pods again we should see two containers in the same pod.

In [ ]:
podman ps -a --pod

We can also use the following command:

In [ ]:
podman pod ps

Let's add the second container, the fronend.

In [ ]:
podman run -d  --pod=my-pod \
  -e WORDPRESS_DB_NAME="wp" \
  -e WORDPRESS_DB_USER="wordpress" \
  -e WORDPRESS_DB_PASSWORD="wordpress" \
  -e WORDPRESS_DB_HOST="127.0.0.1" \
  --name wp-web \
  docker.io/library/wordpress:latest

As you can see we've used the address 127.0.0.1 for the environment variable WORDPRESS_DB_HOST, this means we're pointing to localhost for finding the database. This is only possible because both containers (the web and the database) are running in the same pod.

Checking the logs we can see there is no issue with the database, confirming it works properly:

In [ ]:
podman logs wp-web

Last, I'd like to mention that it's possible to create a systemd unit file or quadlet for a pod. You could do that the same way we did it for a container in previous section of this workshop.

# Deploy Patient Portal application using pods

This time we are going to demonstrate how to deploy the patient portal application using pods. For this we will have 1 pod with the frontend and database containers. As both containers will be in the same pod they share the same network, hence we'll need a single network for all of our application.

![PatientPortalApplication](Pictures/patient-portal-application-pod.png)

Start creating the networks:

In [ ]:
podman network create

# Cleanup

In [ ]:
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f